# PIPELINE
 DATALOADER --> GRAPH CONSTRUCTION --> GNN --> EXPLAIN 

## DataLoader

In [1]:
from torch.utils.data import Dataset
import os 
from PIL import Image

In [2]:
class CUB (Dataset):
    '''
    parts 
        part_locs 
        parts (names)
    images 
        200 classes
    bounding box  (for cropping images)
    classes.txt
    train_test_split.txt
    image_class_labels.txt
    
    '''

    def __init__(self, root_dir, transform = None):
        self.root_dir = root_dir # D:\TorchProject\dataset\cub\CUB_200_2011
        self.transform = transform
        self.image_dir = os.path.join(root_dir,'images')
        self.image_mapping = self._load_image_mapping()
        self.label_mapping = self._load_label_mapping()
        self.part_locs_mapping = self._load_part_locs_mapping()
        self.part_name_mapping = self._load_part_name_mapping()
        self.bounding_box_mapping = self._load_bounding_box()
        #self.classes_name = self._load_class_names()
    
    # def _load_class_names(self):
    #     class_names = {}
    #     classes_file = os.path.join(self.root_dir,'classes.txt')
    #     with open(classes_file, 'r') as file:
    #         for line in file:
    #             class_id, class_name = line.strip().split()
    #             if class_id not in class_names:
    #                 class_names[class_id] = {}
    #             class_names[class_id] = class_name
    #     return class_names

    def _load_bounding_box(self):
        bounding_box = {}
        box_file = os.path.join(self.root_dir, 'bounding_boxes.txt')
        with open(box_file, 'r') as file:
            for line in file:
                image_id, x, y, width, height = line.strip().split()
                bounding_box[image_id] = [float(x), float(y), float(width), float(height)]
        return bounding_box

    def _load_part_name_mapping (self):
        part_names = {}
        part_names_path = os.path.join(self.root_dir, 'parts', 'parts.txt')
        with open(part_names_path, 'r') as file:
            for line in file:
                part_id, part_name = line.strip().split()
                part_names[part_id] = {}
                part_names[part_id] = part_name
        return part_names
              
    def _load_part_locs_mapping (self):
        part_locs_mapping = {}
        part_locs = os.path.join(self.root_dir, 'parts', 'part_locs.txt')
        with open(part_locs, 'r') as file:
            for line in file:
                image_id, part_id, x, y, _ = line.strip().split()
                if image_id not in part_locs_mapping:
                    part_locs_mapping[image_id] = {}
                part_locs_mapping[image_id][part_id] = [float (x), float (y)]
        return part_locs_mapping

    def _load_label_mapping (self):
        label_mapping = {}
        label_file = os.path.join(self.root_dir,'image_class_labels.txt')
        with open(label_file, 'r') as file:
            for line in file:
                image_id, label_id = line.strip().split()
                label_mapping[image_id] = int(label_id)
        return label_mapping

    def _load_image_mapping (self):
        image_mapping = {}
        image_file = os.path.join(self.root_dir,'images.txt')
        with open(image_file, 'r') as file:
            for line in file:
                image_id, image_name = line.strip().split()
                image_mapping[image_id] = image_name
        return image_mapping

    def __len__(self):
        return len(self.image_mapping)

    def __getitem__(self, index):
        label = self.label_mapping[str(index+1)]
        part_locs = self.part_locs_mapping.get(str(index), [])
        image_name = self.image_mapping[str(index)]
        bboxes = self.bounding_box_mapping.get(str(index), [])
        #part_name = self.part_name_mapping.get(str(index),[])
        image= Image.open(os.path.join(self.image_dir,image_name)).convert('RGB')
        
        # Didnt check if this task is done right, because no need to be in here currently 
        # class_names_list= []
        # class_names = self.class_names.get(str(index), [])
        # for int (i) in class_names:
        #     class_names_list.append(class_names[i])
        
        part_name_list = []
        part_name_dict = {}
        for key in part_locs:
            part_name = self.part_name_mapping.get(str(key),[])
            part_name_dict[part_name] = part_locs[key]
            part_name_list.append(part_name)

        if self.transform:
           image = self.transform(image)
#        return (image, label,part_name, part_locs, bboxes, part_name_list, part_name_dict)
        return (image, label, part_locs, bboxes, part_name_dict) #, class_names    # why has to have part_locs to display right the location?
    # dict: 'back': [0.0, 0.0],'beak': [312.0, 182.0], 'belly': [0.0, 0.0], 'breast': [0.0, 0.0], 'crown': [186.0, 45.0],..
    # locs: '8': [0.0, 0.0], '9': [0.0, 0.0], '10': [100.0, 221.0], '11': [183.0, 101.0], '12': [0.0, 0.0], '13': [0.0, 0.0], ..

In [3]:
### for testing

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from torch.utils.data import DataLoader

root_dir = 'D:\TorchProject\dataset\cub\CUB_200_2011'
# run dataset == __init__
dataset = CUB(root_dir) # --> load sample to check
dataloader = DataLoader(dataset, batch_size=2, shuffle=False) #divide dataset by batch
image, label, part_locs, bboxes, part_name_dict = dataset[1]

print(dataset[1])

(<PIL.Image.Image image mode=RGB size=500x335 at 0x21F2BE433D0>, 1, {'1': [0.0, 0.0], '2': [312.0, 182.0], '3': [0.0, 0.0], '4': [0.0, 0.0], '5': [186.0, 45.0], '6': [247.0, 79.0], '7': [0.0, 0.0], '8': [0.0, 0.0], '9': [0.0, 0.0], '10': [100.0, 221.0], '11': [183.0, 101.0], '12': [0.0, 0.0], '13': [0.0, 0.0], '14': [0.0, 0.0], '15': [215.0, 194.0]}, [60.0, 27.0, 325.0, 304.0], {'back': [0.0, 0.0], 'beak': [312.0, 182.0], 'belly': [0.0, 0.0], 'breast': [0.0, 0.0], 'crown': [186.0, 45.0], 'forehead': [247.0, 79.0], 'left_eye': [0.0, 0.0], 'left_leg': [0.0, 0.0], 'left_wing': [0.0, 0.0], 'nape': [100.0, 221.0], 'right_eye': [183.0, 101.0], 'right_leg': [0.0, 0.0], 'right_wing': [0.0, 0.0], 'tail': [0.0, 0.0], 'throat': [215.0, 194.0]})


### Visualize

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from torch.utils.data import DataLoader

def visualize_bounding_box(image, part_name_dict, bboxes):
    # Create figure and axes
    fig, ax = plt.subplots(1)

    # Display the image
    ax.imshow(image)

    # Extract bounding box coordinates
    x, y, width, height = bboxes
    rect_pb = patches.Rectangle((x, y), width, height, linewidth=1, edgecolor='r', facecolor='none')

    ax.add_patch(rect_pb)
    # Iterate over part bounding boxes and draw rectangles
    for part_id, part_bb in part_name_dict.items():
        x_pb, y_pb = part_bb
        width_pb, height_pb = 5, 5  # Modify the width and height as per your requirement
        rect_pb = patches.Rectangle((x_pb, y_pb), width_pb, height_pb, linewidth=1, edgecolor='b', facecolor='none')
        ax.add_patch(rect_pb)

        # Show the plot
    plt.show()


# Example usage

root_dir = 'D:\TorchProject\dataset\cub\CUB_200_2011'
# run dataser == __init__
dataset = CUB(root_dir) # --> load sample to check
dataloader = DataLoader(dataset, batch_size=2, shuffle=False) #divide dataset by batch

# call len  == __len__
print (len(dataset))

for i in range (1,10):
    image, label, part_locs, bboxes, part_name_dict = dataset[i]
    print (label,part_name_dict, part_locs,bboxes)

    # Visualize all bounding boxes
    visualize_bounding_box(image, part_locs, bboxes)

### Take Class names


In [ ]:
# load the classes for embedding text 

root_dir = 'D:\TorchProject\dataset\cub\CUB_200_2011'

CLASS_NAMES = {}
classes_file = os.path.join(root_dir,'classes.txt')
with open(classes_file, 'r') as file:
    for line in file:
        class_id, class_name = line.strip().split()
        if class_id not in CLASS_NAMES:
            CLASS_NAMES[class_id] = {}
        CLASS_NAMES[class_id] = class_name
print(CLASS_NAMES)

for i in range (1,201):
    print (CLASS_NAMES[str(i)])

### BERT API

In [ ]:
# embedding by BERT
import torch
from transformers import BertTokenizer, BertModel

class TextEmbedder:
    def __init__(self):
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.model = BertModel.from_pretrained('bert-base-uncased')
    
    def embed_text(self, text):
        inputs = self.tokenizer(text, return_tensors='pt', padding=True, truncation=True)
        with torch.no_grad():
            outputs = self.model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)  # Mean pooling over tokens
        return embeddings
    
    
### Test 
# Initialize the TextEmbedder
text_embedder = TextEmbedder()

embedded_class_names = {}
# Example class names
for i in range (1,len(CLASS_NAMES)+1):
#range(1,201):
    embedded_class_names[i] = (text_embedder.embed_text(CLASS_NAMES[str(i)]))

# Print the embeddings
print(embedded_class_names)


In [29]:
# check embedded tensor 

#len(embedded_class_names)
#embedded_class_names[100]
embedded_class_names[1].shape

torch.Size([1, 768])

### Graph Construction
- Node (1 -> 15)
- Node (16 -> 216)
- Edges (init)

- load the class names from classes.txt file
- embedding each class name (each node) 
- put each embedded name into each node (from 16 to 216) 


In [ ]:
### Graph construction
import networkx as nx
import torch
import random

class Graph_construct:
    def __init__(self):
        self.graph = nx.Graph()
  

    def add_nodes(self, part_locs):
        # Add nodes from 1st to 15th with their attributes
        for i in range(1, 16):
            x, y = part_locs[str(i)] # Replace with actual position (x, y) data if available
            title = f"Node {i}"
            image_tensor = torch.rand(16, 16)  # Replace with actual image tensor data if available

            node_attributes = {
                "x": x,
                "y": y,
                "title": title,
                "image": image_tensor ## wait for ViT to split
            }

            self.graph.add_node(i, **node_attributes)

        # Add nodes from 16th to the rest with their class_name attribute
        # num_classes = 200  # Replace with the actual number of classes in the dataset

        text_embedder = TextEmbedder()

        embedded_class_names = {}
        for i in range (1,len(CLASS_NAMES)+1):
            embedded_class_names[i] = (text_embedder.embed_text(CLASS_NAMES[str(i)]))

        for i in range(16, 216):
            node_attributes = {"class_name": embedded_class_names[(i - 15)]}
            self.graph.add_node(i, **node_attributes)

    def calculate_edges(self):
        # Calculate edges for nodes from 1st to 15th based on distance between (x, y) positions
        for i in range(1, 16):
            for j in range(i + 1, 16):
                x1, y1 = self.graph.nodes[i]["x"], self.graph.nodes[i]["y"]
                x2, y2 = self.graph.nodes[j]["x"], self.graph.nodes[j]["y"]
                distance = ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5
                self.graph.add_edge(i, j, weight=distance)

        # Set edges from nodes 16th to the rest to others nodes = 1 for the initial step
        for i in range(16, 216):
            for j in range(1, 16):
                self.graph.add_edge(i, j, weight=1)

if __name__ == "__main__":
    root_dir = 'D:\TorchProject\dataset\cub\CUB_200_2011'
    # run dataser == __init__
    dataset = CUB(root_dir)
    
    graph_construct = Graph_construct()

    graph_construct.add_nodes(dataset[2][2]) # dataset -> image, label, part_locs, 
    graph_construct.calculate_edges()

    # You can access the graph object with all nodes and edges
    G = graph_construct.graph
    print(G.nodes(data=True))
    print(G.edges(data=True))


### Text Embedding

In [ ]:
import torch
from transformers import BertTokenizer, BertModel

class TextEmbedder:
    def __init__(self):
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.model = BertModel.from_pretrained('bert-base-uncased')
    
    def embed_text(self, text):
        inputs = self.tokenizer(text, return_tensors='pt', padding=True, truncation=True)
        with torch.no_grad():
            outputs = self.model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)  # Mean pooling over tokens
        return embeddings

Question: Parts_name should be embedded to put in the node or be used as the label (for level 1: detect the parts of the birds)   